In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cygnus_lib as cy
import mylib as my
from scipy.optimize import curve_fit
from scipy.stats import chisquare
import scipy.stats as stats
from itertools import combinations
import math
%matplotlib inline
import os
import requests
cy.set_atlas_style('square')
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


# LEMON Parameters

In [17]:

# LEMON
#######
x_resolution = y_resolution = 2048
########
#x_resolution = y_resolution = 2304
runI          = [2357, 2359] 
runI          = [2326] 
run_ped       = 2340 
cimax         = 300
cimin         = 0 
dataSelection = 'LTD'
rescale       = 512
nsigma        = 1.5


#########################

try:
    fileoutm = ("./data/run%d_mean.h5" % (run_ped))
    m_image = cy.read_image_h5(fileoutm)
    PedOverMax = m_image[m_image > cimax].size
    print ("Pedestal mean: %.2f, sigma: %.2f, over th. (%d) %d" % 
       (m_image[m_image<cimax].mean(), 
        np.sqrt(m_image[m_image<cimax].var()), cimax,
        (m_image>cimax).sum()))
except:
    print ("No Pedestal file for run %s, run script runs-pedestals.ipynb" % run_ped)
    print ("STOP")

    
try: 
    fileouts = ("./data/run%d_sigma.h5" % (run_ped))
    s_image = cy.read_image_h5(fileouts)
    print ("Sigma mean: %.2f, sigma: %.2f, over th. (50) %d" % 
   (s_image[s_image<50].mean(), 
    np.sqrt(s_image[s_image<50].var()), 
    (s_image>50).sum()))
except:
    print ("No Sigma file for run %s, run script runs-pedestals.ipynb" % run_ped)
    print ("STOP")

#########################

th_image   = np.round(m_image + nsigma*s_image)
print ("light over Th: %.2f " % (th_image.sum()-m_image.sum()))

    
    
for nRi in range(0,len(runI)):
    
    max_image=np.size(cy.swift_listdir(cy.file2FullPathCygnus(dataSelection, runI[nRi],'H5')))
    print ("# of Images Files: %d " % (max_image))
    data_to_save = []
    files = ("./data/dbscan_run%d_cmin_%d_cmax_%d_rescale_%d_nsigma_%.1f_ev_%d_ped_%d.txt" % 
                     (runI[nRi], cimin, cimax, rescale, nsigma, max_image, run_ped))
    for iTr in range(0, max_image):
        if iTr % 10 == 0:
            print ('RUN: ', runI[nRi], 'Event: ', iTr)

        # fii = cy.imageFile2FullPathCygnus(dataSelection, runI[nRi], iTr)
        # filepach = fii[0:35]+'_'+fii[35:]
        # image = cy.swift_read_image_h5(filepach) 
        image = cy.swift_read_image_h5(cy.imageFile2FullPathCygnus(dataSelection, runI[nRi], iTr)) 
        rebin_image     = cy.rebin(image-m_image, (rescale, rescale))  
        rebin_th_image  = cy.rebin((th_image-m_image), (rescale, rescale))

        edges           = (rebin_image > rebin_th_image) & (rebin_image < cimax)         
        points          = np.array(np.nonzero(edges)).T.astype(float) 
        scaler          = StandardScaler()
        X_scaled        = scaler.fit_transform(points)

        dbscan          = DBSCAN(eps=0.05, min_samples = 2)
        dbscan.fit(points)

        clusters = dbscan.fit_predict(X_scaled)

        for ic in range (min(dbscan.labels_), max(dbscan.labels_)):
            ph = 0.
            # print ("value: ", iTr, ic, dbscan.labels_[ic], min(dbscan.labels_), max(dbscan.labels_))
            yc = points[:,1][dbscan.labels_==ic]
            xc = points[:,0][dbscan.labels_==ic]
            ph, dim = cy.cluster_par(yc, xc, rebin_image)
            width, height, pearson = cy.confidence_ellipse_par(yc,xc)
            for j in range(0, dim):
                x=int(xc[j])
                y=int(yc[j])
                #ph += rebin_image[y,x]
                if j == 0:
                    x0start = x
                    y0start = y
            x0end = x
            y0end = y
            data_to_save.append([iTr, ic, dim, ph, ph/dim, 
                                 x0start, y0start, x0end, y0end, width, height, pearson])

    np.savetxt(files, data_to_save, fmt='%.10e', delimiter=" ")
    print ("out file", files)

        


Pedestal mean: 98.41, sigma: 1.30, over th. (300) 24
Sigma mean: 2.56, sigma: 2.12, over th. (50) 15
light over Th: 16136945.54 
# of Images Files: 5400 
RUN:  2326 Event:  0


OSError: Unable to open file (file signature not found)

In [16]:
2304/4

576.0